In [5]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from ultralytics import YOLO  # Requires YOLOv8

# Load YOLOv8 person detector model
yolo_model = YOLO("yolov8n.pt")  # You can use yolov8s.pt or yolov8m.pt for better accuracy

# Load the trained fall detection model
fall_model = load_model("falldown_pose_model.h5")

# Setup MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(static_image_mode=False)

# Labels
class_labels = ['FALL DOWN', 'NOT FALLEN']

# Load video
cap = cv2.VideoCapture("video_example.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape

    # Run YOLO detection (returns person bounding boxes)
    results = yolo_model(frame)[0]
    for box in results.boxes:
        class_id = int(box.cls[0])
        if class_id == 0:  # Class 0 = person
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            person_roi = frame[y1:y2, x1:x2]

            # Convert ROI to RGB
            rgb_roi = cv2.cvtColor(person_roi, cv2.COLOR_BGR2RGB)
            result = pose.process(rgb_roi)

            if result.pose_landmarks:
                mp_drawing.draw_landmarks(person_roi, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                landmarks = []
                for lm in result.pose_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])

                if len(landmarks) == 99:
                    input_data = np.array(landmarks).reshape(1, 99, 1)
                    prediction = fall_model.predict(input_data, verbose=0)[0][0]
                    status = 1 if prediction > 0.5 else 0
                    label = f"{class_labels[status]} ({prediction:.2f})"
                    color = (0, 255, 0) if status == 1 else (0, 0, 255)

                    # Draw bounding box and label on original frame
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Multi-Person Fall Detection", frame)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 1 refrigerator, 164.6ms
Speed: 8.8ms preprocess, 164.6ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 refrigerator, 105.0ms
Speed: 2.5ms preprocess, 105.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 refrigerator, 102.5ms
Speed: 3.4ms preprocess, 102.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 refrigerator, 100.0ms
Speed: 2.0ms preprocess, 100.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 refrigerator, 93.2ms
Speed: 2.3ms preprocess, 93.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 refrigerator, 112.1ms
Speed: 3.4ms preprocess, 112.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 refrigerator, 109.0ms
Speed: 3.2ms preprocess, 109.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384,

In [1]:
#!pip install ultralytics

  Using cached ultralytics-8.3.166-py3-none-any.whl.metadata (37 kB)
  Using cached torch-2.7.1-cp311-cp311-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.22.1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached ultralytics-8.3.166-py3-none-any.whl (1.0 MB)
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/216.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/216.1 MB 1.7 MB/s eta 0:02:10
   ------------------------------